## Script to generate augmented images from given images

In [6]:
import os
import imgaug as ia
from imgaug import augmenters as iaa
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers.merge import concatenate, add
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from PIL import Image

In [2]:
#this method takes in the path of the directory containing images and their respective masks, and returns a 4d array with their respective values
def get_images_to_augment(path, im_height=256, im_width=256):
    # get directory path for unaugmented images
    imgs_path = path + "images_unaugmented"
    print("Retrieving images from " + imgs_path)

    # get directory path for unaugmented masks
    mask_path = aug_path + "masks_unaugmented"
    print("Retrieving masks from " + mask_path)
    
    imgs = os.listdir(imgs_path)
    masks = os.listdir(mask_path)

    # remove .DS_Store, if it exists in either directory
    if ".DS_Store" in imgs:
        imgs.remove('.DS_Store')

    if ".DS_Store" in masks:
        masks.remove('.DS_Store')

    # remove directory .ipynb_checkpoints if it exists
    if ".ipynb_checkpoints" in imgs:
        imgs.remove('.ipynb_checkpoints')

    if ".ipynb_checkpoints" in masks:
        masks.remove('.ipynb_checkpoints')

    # arrays are 4d numpy array of shape (N, height, width, channels)
    im_arr = np.zeros((len(imgs), im_height, im_width, 3), dtype=np.uint8)

    msk_arr = np.zeros((len(masks), im_height, im_width, 1), dtype=np.uint8)
    print('Getting and resizing images ... ')
    for n, id_ in tqdm_notebook(enumerate(imgs), total=len(imgs)):
        #Load images
        img = load_img(imgs_path + '/' + id_)
        image = img_to_array(img)
        image = resize(image, (256, 256, 3), mode='constant', preserve_range=True)

        #Load masks
        msk = load_img(mask_path + '/' + id_, color_mode="grayscale")
        mask = img_to_array(msk)
        mask = resize(mask, (256, 256, 1), mode='constant', preserve_range=True)


        im_arr[n] = image
        msk_arr[n] = mask
    # X now contains the respective values of the pixel intensities in the image
    print('Done!')
  
    return im_arr, msk_arr

    

In [3]:
#get data 
#path = './data/test/'
path = './data/duct_test/new_dataset/'
img_arr, masks_arr = get_images_to_augment(path)
print(img_arr.shape)
print(masks_arr.shape)


Retrieving images from ./data/train/to_augment/images
Retrieving masks from ./data/train/to_augment/masks
Getting and resizing images ... 

Done!
(3, 256, 256, 3)
(3, 256, 256, 1)


## Light augmentation

In [4]:
#light augmentation
ia.seed(1)
#sequence of random augmentations
seq_light = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    #iaa.Sometimes(0.5,
        #   iaa.GaussianBlur(sigma=(0, 0.5))
    #),
    # Strengthen or weaken the contrast in each image.
    iaa.contrast.LinearContrast((0.75, 1.5)),
    
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    #iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),

    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    #iaa.Multiply((0.8, 1.2), per_channel=0.2),

    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (1.0, 1.2), "y": (1.0, 1.2)}

    )
], random_order=True) # apply augmenters in random order

seq_light = seq_light.localize_random_state()
seq_light = seq_light.to_deterministic()

images_aug_light = seq_light.augment_images(img_arr)
masks_aug_light = seq_light.augment_images(masks_arr)

print(images_aug_light[0].shape)
print(masks_aug_light[0].shape)


(256, 256, 3)
(256, 256, 1)


## Heavy Augmentation

In [5]:
#heavy augmentation
#method that heavily augments the images

ia.seed(1)

sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# Define our sequence of augmentation steps that will be applied to every image.
seq_heavy = iaa.Sequential(
    [
        #
        # Apply the following augmenters to most images.
        #
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images

        # crop some of the images by 0-10% of their height/width
        sometimes(iaa.Crop(percent=(0, 0.1))),

        # Apply affine transformations to some of the images
        # - scale to 80-120% of image height/width (each axis independently)
        # - translate by -20 to +20 relative to height/width (per axis)
        # - rotate by -45 to +45 degrees
        # - shear by -16 to +16 degrees
        # - order: use nearest neighbour or bilinear interpolation (fast)
        # - mode: use any available mode to fill newly created pixels
        #         see API or scikit-image for which modes are available
        # - cval: if the mode is constant, then use a random brightness
        #         for the newly created pixels (e.g. sometimes black,
        #         sometimes white)
        sometimes(iaa.Affine(
            scale={"x": (1.0, 1.2), "y": (1.0, 1.2)},
            #translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            #rotate=(-45, 45),
            #shear=(-16, 16),
            order=[0, 1],
            cval=(0, 255),
            mode=ia.ALL
        )),

        #
        # Execute 0 to 5 of the following (less important) augmenters per
        # image. Don't execute all of them, as that would often be way too
        # strong.
        #
        iaa.SomeOf((0, 5),#(0,3),
            [
                # Convert some images into their superpixel representation,
                # sample between 20 and 200 superpixels per image, but do
                # not replace all superpixels with their average, only
                # some of them (p_replace).
                sometimes(
                    iaa.Superpixels(
                        p_replace=(0, 1.0),
                        n_segments=(20, 200)
                    )
                ),

                # Blur each image with varying strength using
                # gaussian blur (sigma between 0 and 3.0),
                # average/uniform blur (kernel size between 2x2 and 7x7)
                # median blur (kernel size between 3x3 and 11x11).
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)),
                    iaa.AverageBlur(k=(2, 7)),
                    iaa.MedianBlur(k=(3, 11)),
                ]),

                # Sharpen each image, overlay the result with the original
                # image using an alpha between 0 (no sharpening) and 1
                # (full sharpening effect).
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),

                # Same as sharpen, but for an embossing effect.
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),

                # Search in some images either for all edges or for
                # directed edges. These edges are then marked in a black
                # and white image and overlayed with the original image
                # using an alpha of 0 to 0.7.
                sometimes(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0, 0.7)),
                    iaa.DirectedEdgeDetect(
                        alpha=(0, 0.7), direction=(0.0, 1.0)
                    ),
                ])),

                # Add gaussian noise to some images.
                # In 50% of these cases, the noise is randomly sampled per
                # channel and pixel.
                # In the other 50% of all cases it is sampled once per
                # pixel (i.e. brightness change).
                iaa.AdditiveGaussianNoise(
                    loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                ),

                # Either drop randomly 1 to 10% of all pixels (i.e. set
                # them to black) or drop them on an image with 2-5% percent
                # of the original size, leading to large dropped
                # rectangles.
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5),
                    iaa.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    ),
                ]),

                # Invert each image's channel with 5% probability.
                # This sets each pixel value v to 255-v.
                iaa.Invert(0.05, per_channel=True), # invert color channels

                # Add a value of -10 to 10 to each pixel.
                iaa.Add((-10, 10), per_channel=0.5),

                # Change brightness of images (50-150% of original value).
                iaa.Multiply((0.5, 1.5), per_channel=0.5),

                # Improve or worsen the contrast of images.
                iaa.contrast.LinearContrast((0.5, 2.0), per_channel=0.5),

                # Convert each image to grayscale and then overlay the
                # result with the original with random alpha. I.e. remove
                # colors with varying strengths.
                iaa.Grayscale(alpha=(0.0, 1.0)),

                # In some images move pixels locally around (with random
                # strengths).
                sometimes(
                    iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
                ),

                # In some images distort local areas with varying strength.
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05)))
            ],
            # do all of the above augmentations in random order
            random_order=True
        )
    ],
    # do all of the above augmentations in random order?
    random_order=False
)

seq_heavy.to_deterministic()
images_aug_heavy = seq_heavy.augment_images(img_arr)
masks_aug_heavy = seq_heavy.augment_images(masks_arr)

In [5]:
#augment the images
#for light aug
aug_img = images_aug_light#images_aug_heavy
aug_masks = masks_aug_light#masks_aug_heavy
#heavy_augments = augment_images_heavy(img_arr)
print(aug_masks[0].shape)


(256, 256, 1)


In [7]:
#check if directory to save the files to exists, if not create one
aug_img_path = './data/duct_test/new_dataset/augmented_images'
aug_mask_path = './data/duct_test/new_dataset/augmented_masks'
if not os.path.isdir(aug_img_path):
    os.mkdir(aug_img_path)
if not os.path.isdir(aug_mask_path):
    os.mkdir(aug_mask_path)

#attain the original filenames
ori_imgs = os.listdir('./data/duct_test/new_dataset/augmented_images')
ori_masks = os.listdir('./data/duct_test/new_dataset/augmented_masks')

# save the augmented files with the same filenames in the new folder
# saving the augmented originals

for i in range(len(aug_img)):
    #use PIL to create an img
    augmented = Image.fromarray(aug_img[i], 'RGB')
    #match it to the corresponding name
    img_name = ori_imgs[i]
    #save the image to the directory
    augmented.save(aug_img_path + '/' + img_name)

# saving the augmented masks
aug_masks2 = np.reshape(aug_masks, (len(aug_masks), 256, 256))
for i in range(len(aug_masks2)):
    augmented = Image.fromarray(aug_masks2[i], 'L')
    mask_name = ori_masks[i]
    augmented.save(aug_mask_path + '/' + mask_name)

